In [1]:
#Import files
import datetime
import pandas as pd
from calendar import monthrange
import numpy as np
from pandas.tseries.offsets import MonthEnd
import csv
import os
import warnings; warnings.filterwarnings('ignore')
import pantab
from tableauhyperapi import TableName
from glob import glob
import sqlite3
import pyodbc
import sqlalchemy as sal
from sqlalchemy import create_engine

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\tableau.server\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Import the data
os.chdir('D:\\Survey Data\\Facebook\\')

# add dataframe in next line every month 

In [3]:
# Reading all the data files
df = pd.read_csv("1_CookieCutter\\Data\\Single.csv",encoding='cp1252').astype(str)

In [4]:
df.columns

Index(['record', 'Date', 'Weight', 'uuid', 'dStatus', 'Hid_Age', 'R2',
       'DV_CompanyType', 'DV_AgencyType', 'DV_AgencyBIG6', 'R7',
       'DV_Companysize', 'P12', 'R12', 'DV_media_spend',
       'DV_business_size_media_spend', 'R16', 'R17', 'R18', 'R19', 'R20',
       'LIKEABILITY_Q1_Lr1', 'OE_LIKED_Q2A_Lr1', 'OE_DISLIKED_Q2B_Lr1',
       'OE_MAIN_IDEA_Q3_Lr1', 'CLARITY_Q5_Lr1', 'RELEVANCE_Q6_Lr1',
       'DIFFERENTIATION_Q7_Lr1', 'BELIEVABILITY_Q8_Lr1',
       'BRAND_ATTRIBUTION_Q10_Lr1', 'OE_BRAND_ATT_Q11_Lr1'],
      dtype='object')

In [5]:
type(df['OE_MAIN_IDEA_Q3_Lr1'])

pandas.core.series.Series

# add df in below line every month

In [6]:
# append all DataFrames into one before writing to single table in Hyper
final_df = df.reset_index(drop=True)

# Sentiment scores

In [7]:
# function to print sentiments
# of the sentence.
def sentiment_scores(sentence):

    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()

    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)

In [8]:
sia = SentimentIntensityAnalyzer()

In [9]:
final_df['Q3Lr1_compound'] = [sia.polarity_scores(x)['compound'] for x in final_df['OE_MAIN_IDEA_Q3_Lr1']]
final_df['Q11Lr1_compound'] = [sia.polarity_scores(x)['compound'] for x in final_df['OE_BRAND_ATT_Q11_Lr1']]

In [10]:
final_df.loc[final_df['Q3Lr1_compound'] >= 0.05, 'Q3Lr1_sent'] = 'Positive' 
final_df.loc[final_df['Q3Lr1_compound'] <= -0.05, 'Q3Lr1_sent'] = 'Negative'
final_df.loc[(final_df['Q3Lr1_compound'] >= -0.05) & (final_df['Q3Lr1_compound'] < 0.05), 'Q3Lr1_sent'] = 'Neutral' 

In [11]:
final_df.loc[final_df['Q11Lr1_compound'] >= 0.05, 'Q11Lr1_sent'] = 'Positive' 
final_df.loc[final_df['Q11Lr1_compound'] <= -0.05, 'Q11Lr1_sent'] = 'Negative'
final_df.loc[(final_df['Q11Lr1_compound'] >= -0.05) & (final_df['Q11Lr1_compound'] < 0.05), 'Q11Lr1_sent'] = 'Neutral' 

# to SQL

In [12]:
engine = sal.create_engine('mssql+pyodbc://BASRV10\SQLEXPRESS/CookieCutter?driver=SQL Server?Trusted_Connection=yes')
# establishing the connection to the databse using engine as an interface
conn = engine.connect()

In [13]:
final_df.to_sql('Single_w1', con=engine, if_exists='append',index=False)

In [14]:
final_df

,record,Date,Weight,uuid,dStatus,Hid_Age,R2,DV_CompanyType,DV_AgencyType,DV_AgencyBIG6,...,CLARITY_Q5_Lr1,RELEVANCE_Q6_Lr1,DIFFERENTIATION_Q7_Lr1,BELIEVABILITY_Q8_Lr1,BRAND_ATTRIBUTION_Q10_Lr1,OE_BRAND_ATT_Q11_Lr1,Q3Lr1_compound,Q11Lr1_compound,Q3Lr1_sent,Q11Lr1_sent
0,34_w1,06-26-2021,1,t250mgajd4xhg5jw,Complete,35 to 44 years,Pennsylvania[E],Advertiser,nan,nan,...,Neither easy nor difficult to understand,Somewhat not relevant,Not very new and different,Somewhat not believable,Google,Google delivers a broad digital experience and...,0.4215,0.0000,Positive,Neutral
1,36_w1,06-26-2021,1,fm7bxk6q78ey8fzd,Complete,25 to 34 years,Virginia[E],Advertiser,nan,nan,...,Somewhat difficult to understand,Somewhat not relevant,Not at all new and different,Neither believable nor not believable,Yahoo!,Both are a bit out of touch,0.8591,0.0000,Positive,Neutral
2,37_w1,06-26-2021,1,f2d0gb9nvnsecm34,Complete,65 years+,Texas,Advertiser,nan,nan,...,Extremely easy to understand,Extremely relevant,Very new and different,Extremely believable,Google,It meshes with their operating philosophy.,0.6908,0.0000,Positive,Neutral
3,39_w1,06-26-2021,1,kuvgwbqwp44mcx6z,Complete,35 to 44 years,Ohio,Advertiser,nan,nan,...,Neither easy nor difficult to understand,Somewhat relevant,Neither new and different nor not new and diff...,Somewhat believable,Amazon,They are always evolving,0.0000,0.0000,Neutral,Neutral
4,40_w1,06-26-2021,1,mh28k5ftwnt14nmn,Complete,55 to 64 years,Connecticut,Advertiser,nan,nan,...,Extremely easy to understand,Extremely relevant,Very new and different,Extremely believable,Amazon,Amazon is on the forefront of just about every...,0.4019,0.1779,Positive,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,773_w1,06-26-2021,1,sf56xnzv9p93ggwu,Complete,35 to 44 years,Pennsylvania[E],Advertiser,nan,nan,...,Somewhat easy to understand,Somewhat relevant,Neither new and different nor not new and diff...,Somewhat believable,Facebook,I don’t know,0.4019,0.0000,Positive,Neutral
203,787_w1,06-26-2021,1,ugtrfzrj5qz5d0p2,Complete,25 to 34 years,Wisconsin,Agency,Media,Independent,...,Somewhat easy to understand,Somewhat relevant,Neither new and different nor not new and diff...,Somewhat believable,Facebook,It is suitable with Facebook,0.4215,0.0000,Positive,Neutral
204,818_w1,06-26-2021,1,b42dv1zseg0vuxnk,Complete,35 to 44 years,Florida,Advertiser,nan,nan,...,Somewhat easy to understand,Somewhat relevant,Very new and different,Somewhat believable,Google,Just sounds like they would come up with that,0.4215,0.3612,Positive,Positive
205,901_w1,06-26-2021,1,hhyezuakqqfq89ud,Complete,25 to 34 years,Ohio,Advertiser,nan,nan,...,Somewhat difficult to understand,Somewhat relevant,Not very new and different,Neither believable nor not believable,Amazon,They use a lot of the same jargon that I've se...,0.8225,0.5574,Positive,Positive
